In [2]:
!git clone https://github.com/neccam/slt.git

Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.


In [3]:
%cd slt

/content/slt


In [4]:
!chmod +x data/download.sh
!data/download.sh

--2021-01-03 20:09:15--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  21.4MB/s    in 1m 54s  

2021-01-03 20:11:09 (16.1 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2021-01-03 20:11:09--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130831227 (125M)
Saving to: ‘phoenix14t.pami0.dev’

phoenix14t.pami0.de 100%[===================>] 124.77M  17.8MB/s    in 8.5s    

2021-01-03 20:11:18 (14.7 MB/s) - ‘phoenix14t.pami0.dev’ saved [1308312

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp ../drive/MyDrive/slt/requirements.txt requirements.txt

In [6]:
!pip -q install -r requirements.txt

     |████████████████████████████████| 112kB 11.8MB/s 
     |████████████████████████████████| 112kB 15.3MB/s 
     |████████████████████████████████| 163kB 18.4MB/s 
     |████████████████████████████████| 399kB 25.8MB/s 
     |████████████████████████████████| 2.3MB 30.3MB/s 
     |████████████████████████████████| 81kB 10.0MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 2.4MB 75.3MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 296kB 74.4MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 92kB 12.1MB/s 
     |████████████████████████████████| 92kB 12.0MB/s 
     |████████████████████████████████| 13.1MB 59.3MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 20.2MB 368kB/s 
     |

In [7]:
!mkdir data/PHOENIX2014T 

!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

!unzip -q '../drive/MyDrive/slt/keypoints/train_body_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/train_face_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/train_hand_pose3d.zip' -d data

!unzip -q '../drive/MyDrive/slt/keypoints/test_keypoints.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/dev_keypoints.zip' -d data

In [8]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

body_parts = ['body_pose3d',
              'face_pose3d',
              'hand_pose3d']

for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  src_features = load_dataset_file(filename)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami1.{}".format(s)
  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'
    
    new_features_list = []

    for body_part in body_parts:
      dst = os.path.join(os.path.join(wd, s), os.path.join(body_part, name))

      new_features = np.load(dst)
      new_features = new_features.reshape((new_features.shape[0], np.prod(new_features.shape[1: ])))
      new_features_list.append(new_features)
      
    
    new_features = np.concatenate(new_features_list, axis=1)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [00:14<00:00, 504.56it/s]


In [9]:
import yaml
with open("configs/sign.yaml") as f:
     list_doc = yaml.load(f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
for s in ['train', 'test', 'dev']:
  list_doc['data'][s] = ['PHOENIX2014T/phoenix14t.pami{}.{}'.format(i, s) for i in range(2)]

list_doc['data']['feature_size'] = [1024, 417]
list_doc['training']['recognition_loss_weight'] = 40

In [11]:
with open("configs/sign.yaml", "w") as f:
    yaml.dump(list_doc, f)

In [12]:
!python -m signjoey train configs/sign.yaml

2021-01-03 20:16:49.675445: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-01-03 20:16:53.301997: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-03 20:16:53.304709: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-03 20:16:53.356487: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-03 20:16:53.357126: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.75GiB deviceMemoryBandwidth: 836.37GiB/s
2021-01-03 20:16:53.357190: I tensorflow/stream_executor/platform/defau